# Awesomenauts Web Scraping

## Import the libraries that we are gonna use...

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json

## Useful Links
* https://awesomenauts.fandom.com/wiki/Awesomenauts_Wiki
* https://awesomenauts.fandom.com/wiki/Anchor_Hook
* https://awesomenauts.fandom.com/wiki/Admiral_Swiggins
* https://codeamaze.com/web-viewer/json-explorer
* http://jsonviewer.stack.hu/
* https://pypi.org/project/json2html/


## Functions
* parse_character
* parse_skill
* parse_upgrades

## Some basic web scraping

In [9]:
URL = "https://awesomenauts.fandom.com/wiki/Admiral_Swiggins"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
soup.title.string

'Admiral Swiggins - Official Awesomenauts Wiki'

In [11]:
soup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Admiral Swiggins - Official Awesomenauts Wiki\n  </title>\n  <script>\n   document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );\n  </script>\n  <script>\n   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Admiral_Swiggins","wgTitle":"Admiral Swiggins","wgCurRevisionId":46229,"wgRevisionId":46229,"wgArticleId":8116,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using DynamicPageList parser tag","Fighter","Stun","Blind","Shield","Tether","Amplify Damage","Slow","Abilities","PPT","Med-i\'-Can","Solar Tree","Piggy Bank","Space Air Max","Baby Kuri Mammoth","Admiral Swiggins","Tips \\u0026 Tactics","Awesomenauts","Damage Over 

# Functions for scraping Awesomenauts data
* Skill upgrades
* Skills
* Whole characters

In [12]:
def parse_upgrade(upgrade_html, _print = True):
    
   
    # Get the upgrade name and the URL for its image
    upgrade_name = upgrade_html.a['title']
    upgrade_img = upgrade_html.find("td").find("img").get("src")
    upgrade_img = upgrade_img[:upgrade_img.lower().find(".png")+4]

    # Parse the html table into a data frame
    df = pd.read_html(upgrade_html.prettify())[0]

    # Pull out upgrade description and cost
    upgrade_description=df.loc[1,1].split(":")[1].strip()
    cost = df.loc[0,2]
    
    # Parse the upgrade stats
    u = df.loc[1:,2:].T
    u.columns = u.iloc[0]
    u = u.drop(u.index[0])
    u = u.dropna(axis=0, how="all")
    u = u.dropna(axis=1, how="all")
    u = u.reset_index(drop=True)
    
    # Build a dictionary of the upgrade data
    upgrade_dict = {"name":upgrade_name,
                    "img":upgrade_img,
                   "description":upgrade_description,
                   "cost":cost}
    
    if _print:
        print(f"Name:{upgrade_name} (Cost={cost})\nDescription:{upgrade_description}")
    
    # List of upgrade levels
    upgrade_levels = []
    
    # Loop throught the upgrade levels' data
    for index, row in u.iterrows():
        
        # build a dictionary of each level's upgrade info
        upgrade_level_details = {}
        for property in row.index:
            upgrade_level_details[property] = row[property]
            if _print:
                print(f"\t{index}:{property}:{row[property]}")
            
        upgrade_levels.append(upgrade_level_details)
    
    # Store the details of the upgrade levels in the main upgrade dictionary
    upgrade_dict["levels"] = upgrade_levels
            
            
    return upgrade_dict
    


In [13]:
URL = "https://awesomenauts.fandom.com/wiki/Anchor_Hook"
#URL = "https://awesomenauts.fandom.com/wiki/Sword_Strike"
#URL = "https://awesomenauts.fandom.com/wiki/Mighty_Throw"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all of the tables on the page
tables = soup.find_all("table", class_="framed-table")
skill_html = tables[0]

upgrade_html = tables[3]

upgrade_html.find("td").find("img").get("src")
upgrade_img = upgrade_html.find("td").find("img").get("src")
upgrade_img = upgrade_img[:upgrade_img.lower().find(".png")+4]


print(upgrade_img)


# Parse the html table into a data frame
df = pd.read_html(skill_html.prettify())[0].T
df = df.dropna(axis=1, how="all")

# Parse the df for the skill name and description
skill_name = df.iloc[0,1].split("[")[0].strip()
skill_description = df.iloc[0,2]


# Create a new df of just the skill stats
stats = df.loc[:,3:]
stats.columns = stats.iloc[0]
stats = stats.drop(stats.index[0])
stats = stats.reset_index(drop=True)
stats.iloc[0]


#stats.loc[0,"Attack speed"]
#stats.loc[0,"Slash Damage"]
#type(stats.loc[0,"Attack speed"])

df

https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/d/de/Shop_icons_captain_skill_b_upgrade_c.png


,0,1,2,3,4,5,6,7
0,NaN,Anchor Hook [edit],"Swiggins hurls Homboldt forward, and grapples ...",Damage,Droid damage,Stun Duration,Cooldown,Range
1,Anchor Hook [edit] Swiggins hurls Homboldt f...,Anchor Hook [edit],"Swiggins hurls Homboldt forward, and grapples ...",310 (486.7),100 (157),0.25s,7s,11


In [14]:
URL = "https://awesomenauts.fandom.com/wiki/Anchor_Hook"
#URL = "https://awesomenauts.fandom.com/wiki/Sword_Strike"
#URL = "https://awesomenauts.fandom.com/wiki/Mighty_Throw"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all of the tables on the page
tables = soup.find_all("table", class_="framed-table")
skill_html = tables[0]
skill_img = skill_html.find("img").get("src")
skill_img = skill_img[:skill_img.lower().find(".png")+4]
skill_img


'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/0/0b/UI_Skillbutton_Captain_AnchorHook.png'

In [15]:
def parse_skill(skill_html, _print=True):
    '''
    In: HTML of an awesomenauts character skill
    Out: Dictionary of the details of the skill
    Process:  Use pandas to convert the HTML to a dataframe and then wrangle the data into a dictionary
    '''

    # Get the skill imagine URL
    skill_img = skill_html.find("img").get("src")
    skill_img = skill_img[:skill_img.lower().find(".png")+4]
    
    # Parse the html table into a data frame
    df = pd.read_html(skill_html.prettify())[0].T
    df = df.dropna(axis=1, how="all")
    
    # Parse the df for the skill name and description
    skill_name = df.iloc[0,1].split("[")[0].strip()
    skill_description = df.iloc[0,2]

    # Create a new df of just the skill stats
    stats = df.loc[:,3:]
    stats.columns = stats.iloc[0]
    stats = stats.drop(stats.index[0])
    stats = stats.reset_index(drop=True)
    stats.iloc[0]
    
    # Create a dictionary to store all of the skill details
    skill_dict = {"name":skill_name,
                 "description":skill_description,
                 "img":skill_img}

    if _print:
        print(f"Skill:{skill_name}\n{skill_description}")
    
    # Add the skill stats and values to the skill dictionary
    skill_stats_dict = {}
    for stat in stats.columns:
        matching_stats = stats.loc[0,stat]
        
        # If more than one stats with the same name...
        if type(matching_stats) == pd.Series:
            for i, v in enumerate(matching_stats):
                skill_stats_dict[f"{stat.strip()}-{i}"] = v
                if _print:
                    print(f"\t{stat.strip()}-{i}={v}")
        else:
            if _print:
                print(f"\t{stat}={stats.loc[0,stat]}")
            skill_stats_dict[str(stat).strip()] = stats.loc[0,stat]
                
    skill_dict["stats"] = skill_stats_dict
        
    # Get the full details of the skill
    skill_url_name = skill_name.replace(" ","_")
    skill_url=f"https://awesomenauts.fandom.com/wiki/{skill_url_name}"
    page = requests.get(skill_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find all of the tables on the page
    tables = soup.find_all("table", class_="framed-table")
    
    # Loop thorugh each skill upgrade tables...
    if _print:
        print("### Skill Upgrades ###")    
    
    upgrades = []
    
    # Build a list of upgrade details for this skill
    for i in range(2, len(tables)):
        new_upgrade = parse_upgrade(tables[i], _print)
        upgrades.append(new_upgrade)
    
    skill_dict["upgrades"] = upgrades

    
    return skill_dict
        
    
    

In [26]:
def parse_character(character_html, _print = True):
    
   
    # Get the chracter's name from the page title
    character_name = character_html.title.string.split("-")[0].strip()

    # Get the character's image file
    img = soup.find(class_="Awesomenaut-Portrait").find("img").get("src")
    img = img[:img.lower().find(".png")+4]
    
    # Find all of the tables in the HTML
    tables = character_html.find_all("table", class_="framed-table")

    # Parse the html table into a data frame
    df = pd.read_html(tables[0].prettify())[0]
    df = df.dropna(axis=0, how="all")
   
    # Create a data frame for the character's abilities
    types_of_abilities = df.iloc[-1,0].split(":")[1].strip()
    abilities = [x.strip() for x in types_of_abilities.split(",")]
    
    # Create a dataframe for the character's stats
    stats = df[1:-2]
    stats = stats.dropna(axis=0, how="all")
    stats = stats.dropna(axis=1, how="all")
    stats = stats.T
    stats.columns = stats.iloc[0]
    stats = stats.drop(stats.index[0])
    stats = stats.reset_index(drop=True)

    
    # Print the characters name, abilities and stats
    if _print:
        print(f"Name:{character_name}\nAbilities:{abilities}")
    
    character_dict = {"name":character_name,
                      "img":img,
                     "abilities":abilities}
    
    for stat in stats.columns:
        stat_name = stat.split(":")[0].strip()
        character_dict[stat_name] = stats.loc[0,stat]
        
        if _print:
            print(f"\t{stat_name}:{stats.loc[0,stat]}")

    # Loop throuth the HTML tables that have the details for each skill...
    if _print:
        print("### Skills ###")
    
    skills=[]
    
    for i in range(1, len(tables)-1, 2):
        new_skill = parse_skill(tables[i], _print)
        skills.append(new_skill)
        

    character_dict["skills"] =skills
    
    return character_dict
        


In [17]:
# Testing parsing an individual Skill

URL = "https://awesomenauts.fandom.com/wiki/Anchor_Hook"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

tables = soup.find_all("table", class_="framed-table")

#print(tables[0].prettify())
skill = parse_skill(tables[0])

        
#print(json.dumps(skill,indent=4, separators=(", ", " = ")))

print(json.dumps(skill))

Skill:Anchor Hook
Swiggins hurls Homboldt forward, and grapples towards whatever he hits. If you thought getting hit by an anchor is bad, wait for the iron-clad squid that's coming after it!
	Damage=310 (486.7)
	Droid damage=100 (157)
	Stun Duration=0.25s
	Cooldown=7s
	Range=11
### Skill Upgrades ###
Name:Flying Fish Compass (Cost=190)
Description:Reduces the cooldown of anchor hook  In the winter they point south, in the summer they point north, easy!
	0:Upgrade:Lv1
	0:Cooldown:-2s
Name:Blue Heart Medal (Cost=165)
Description:Increases the base damage of anchor hook  Draw me like one of your French girls.
	0:Upgrade:Lv1
	0:Base damage:+16%
	0:Base droid damage:+28%
	1:Upgrade:Lv2
	1:Base damage:+32%
	1:Base droid damage:+56%
Name:Mobile Periscope (Cost=185)
Description:Increases the range of anchor hook  Penguins on iceplanet Rill use these to find flying fish.
	0:Upgrade:Lv1
	0:Range:+4
Name:Treasure Lottery Map (Cost=175)
Description:Enables a shield when hitting an enemy Awesomenau

In [18]:
# Test parsing a whole character

URL = "https://awesomenauts.fandom.com/wiki/Admiral_Swiggins"
URL = "https://awesomenauts.fandom.com/wiki/Deadlift"

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

character_dict = parse_character(soup)

#print(json.dumps(character_dict,indent=4, separators=(", ", " = ")))
print(json.dumps(character_dict))

Name:Deadlift
Abilities:['Area of Effect', 'Knockback', 'Shield', 'Healing', 'Weaken', 'Empower', 'Slow', 'Stun']
	Health:1600 (2816)
	Movement Speed:7.46
	Attack Type:Melee
	Role:Support
	Mobility:Balanced
### Skills ###
Skill:Power Lunge
In true Bovinian fashion, lunge into enemies and cause an explosive burst that deals damage.
	Damage=300 (471)
	Cooldown=7s
### Skill Upgrades ###
Name:Red Asteroid Horn Polish (Cost=135)
Description:Reduces the cooldown of power lunge.  A brand of horn polish set up by the famous Bovinion hero
	0:Upgrade:Lv1
	0:Cooldown:-0.8s
	1:Upgrade:Lv2
	1:Cooldown:-1.6s
Name:Muscular-Posing Oil (Cost=150)
Description:Increases the range of power lunge.  Made for intergalactic posing contests, but can also be used to oil robots and vikings.
	0:Upgrade:Lv1
	0:Range:+25%
Name:Holotech Kickboxing Gloves (Cost=160)
Description:Increases the damage of power lunge.  Makes your knuckles as powerful as a bull ram and protects them from blasts as strong as dynamite.
	0:U

In [19]:


# Test parsing a whole character

URL = "https://awesomenauts.fandom.com/wiki/Admiral_Swiggins"
URL = "https://awesomenauts.fandom.com/wiki/Deadlift"

page = requests.get(URL)
character_html = BeautifulSoup(page.content, 'html.parser')

# Get the chracter's name from the page title
character_name = character_html.title.string.split("-")[0].strip()

# Find all of the tables in the HTML
tables = character_html.find_all("table", class_="framed-table")

# Parse the html table into a data frame
df = pd.read_html(tables[0].prettify())[0]
df = df.dropna(axis=0, how="all")

# Create a data frame for the character's abilities
types_of_abilities = df.iloc[-1,0].split(":")[1].strip()
abilities = [x.strip() for x in types_of_abilities.split(",")]

## Get list of all Awesomenauts

In [20]:
# Parse Awesomenauts wiki for a list of URLs to the individual character pages
URL = "https://awesomenauts.fandom.com"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

nauts_nav = soup.find_all("div", class_="portal")
nauts_list = nauts_nav[1].find_all("li")

nauts_urls = {}
for naut in nauts_list:
    href = f"{URL}/{naut.a.get('href')}"
    nauts_urls[naut.string] = href
    
print(nauts_urls)

{'Admiral Swiggins': 'https://awesomenauts.fandom.com//wiki/Admiral_Swiggins', 'Ayla': 'https://awesomenauts.fandom.com//wiki/Ayla', 'Chucho Krokk': 'https://awesomenauts.fandom.com//wiki/Chucho_Krokk', 'Clunk': 'https://awesomenauts.fandom.com//wiki/Clunk', 'Coco Nebulon': 'https://awesomenauts.fandom.com//wiki/Coco_Nebulon', 'Commander Rocket': 'https://awesomenauts.fandom.com//wiki/Commander_Rocket', 'Deadlift': 'https://awesomenauts.fandom.com//wiki/Deadlift', 'Derpl Zork': 'https://awesomenauts.fandom.com//wiki/Derpl_Zork', 'Dizzy': 'https://awesomenauts.fandom.com//wiki/Dizzy', 'Froggy G': 'https://awesomenauts.fandom.com//wiki/Froggy_G', 'Genji the Pollen Prophet': 'https://awesomenauts.fandom.com//wiki/Genji_the_Pollen_Prophet', 'Gnaw': 'https://awesomenauts.fandom.com//wiki/Gnaw', 'Ix the Interloper': 'https://awesomenauts.fandom.com//wiki/Ix_the_Interloper', 'Jimmy And The Lux5000': 'https://awesomenauts.fandom.com//wiki/Jimmy_And_The_Lux5000', 'Ksenia': 'https://awesomenauts

In [21]:
import random

In [25]:
# Pick a random Awesomenaut page, scrape the details and print JSON data
URL= "https://awesomenauts.fandom.com//wiki/Admiral_Swiggins"
URL="https://awesomenauts.fandom.com//wiki/Derpl_Zork"
URL = 'https://awesomenauts.fandom.com//wiki/Voltar_the_Omniscient'
#URL = random.choice(list(nauts_urls.values()))
print(URL)
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

img = soup.find(class_="Awesomenaut-Portrait").find("img").get("src")
img = img[:img.lower().find(".png")+4]

img

#character_dict = parse_character(soup, _print=True)

#print(json.dumps(character_dict,indent=4, separators=(", ", " = ")))
#print(json.dumps(character_dict.get("skills")))

#character_dict




https://awesomenauts.fandom.com//wiki/Voltar_the_Omniscient


'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/3/31/Voltar_Alpha.png'

In [23]:
skills = character_dict.get("skills")
for skill in skills:
    try:
        json.dumps(skill)
    except:
        break
        
print(skill)
json.dumps(skill)
u = skill.get("upgrades")

for k,v in skill.items():
    print(f"{k}:{v}")
    json.dumps(v)

{'name': 'Bovinian Lift Off', 'description': 'Deadlift uses his powerful legs, trained extensively on the legpress, to launch himself in the air.', 'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/a/af/UI_Skillbutton_Crumple_Jump.png', 'stats': {'Jumps': '1'}, 'upgrades': [{'name': 'Power Pills Turbo', 'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/5/5f/Upgrade_Power_pills_turbo.png', 'description': 'Increases maximum health.  Insert pill into rear end of digestive tract.', 'cost': '130', 'levels': [{'Upgrade': 'Lv1', 'Health': '+8%'}, {'Upgrade': 'Lv2', 'Health': '+16%'}]}, {'name': "Med-i'-can", 'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/4/40/Upgrade_Med-i%27-can.png', 'description': 'Automatically regenerate health.  Hello... anyone there? Please get me out of here!!!', 'cost': '90', 'levels': [{'Upgrade': 'Lv1', 'Regeneration': '+90%'}, {'Upgrade': 'Lv2', 'Regeneration': '+180%'}]}, {'name': 'Space Air Max

## Create a JSON file of all Awesomenauts data

In [128]:
awesome_nauts = {}
for name, url in nauts_urls.items():
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    try:
        character_dict = parse_character(soup, _print=True)
    
        awesome_nauts[name] = character_dict
        
        json.dumps(character_dict)
    except:
        print("ERRRRORRR" + name)
        break
    
    

Name:Admiral Swiggins
Abilities:['Stun', 'Blind', 'Shield', 'Tether', 'Amplify Damage', 'Slow']
	Health:1500 (2640)
	Movement Speed:7.2
	Attack Type:Melee/Ranged
	Role:Fighter
	Mobility:Tactical
### Skills ###
Skill:Anchor Hook
Swiggins hurls Homboldt forward, and grapples towards whatever he hits. If you thought getting hit by an anchor is bad, wait for the iron-clad squid that's coming after it!
	Damage=310 (486.7)
	Droid damage=100 (157)
	Stun Duration=0.25s
	Cooldown=7s
	Range=11
### Skill Upgrades ###
Name:Flying Fish Compass (Cost=190)
Description:Reduces the cooldown of anchor hook  In the winter they point south, in the summer they point north, easy!
	0:Upgrade:Lv1
	0:Cooldown:-2s
Name:Blue Heart Medal (Cost=165)
Description:Increases the base damage of anchor hook  Draw me like one of your French girls.
	0:Upgrade:Lv1
	0:Base damage:+16%
	0:Base droid damage:+28%
	1:Upgrade:Lv2
	1:Base damage:+32%
	1:Base droid damage:+56%
Name:Mobile Periscope (Cost=185)
Description:Increases

Name:Explosive Neckband (Cost=220)
Description:Increases base damage of chain whack  Let things escalate quickly!
	0:Upgrade:Lv1
	0:Base damage:+14%
	1:Upgrade:Lv2
	1:Base damage:+28%
Name:Prison Guard Keys (Cost=160)
Description:Enemies hit by Chain Whack deal less damage.  Stealthly stolen from a Kremzon prison guard.
	0:Upgrade:Lv1
	0:Damage:-15%
	0:Duration:1.5s
Name:Ion Blowtorch (Cost=160)
Description:Increaes damage of chain whack when enemy Awesomenauts with less than 45% HP are near.  Crème brûlée anyone?
	0:Upgrade:Lv1
	0:Damage:+21%
	1:Upgrade:Lv2
	1:Damage:+42%
Name:Sonic Listening Device (Cost=160)
Description:Mark critters to let them drop more health  Can you hear the Solar bubble?
	0:Upgrade:Lv1
	0:Extra Health:250 (440)
Skill:Rage
Toggle rage mode on/off making Ayla float, and deal heavy damage
	Damage=79 (124.03) (+15% per target)
	Attack Speed=209.6
	Self damage=40 (62.8)
	Size=5.6
	Shield=10% (+% per target)
### Skill Upgrades ###
Name:Angry Drawings (Cost=220)
Desc

Name:Neon Nitro Muffler (Cost=160)
Description:Adds an exhaust flame burst to bike when starting Nitro Boost.  They will hear you coming from light years away!
	0:Upgrade:Lv1
	0:Damage over time:150 (235.5)
	0:Damage duration:4s
	1:Upgrade:Lv2
	1:Damage over time:300 (471)
	1:Damage duration:4s
Name:BOOM Gas (Cost=165)
Description:Causes a damaging explosion when manually detonating your bike. Also halves the cooldown of the manual detonation.  Cheap AND highly flammable. Available at your local space gas station.
	0:Upgrade:Lv1
	0:Damage:350 (549.5)
	0:Cooldown:-50%
Name:Anti-Zork Windscreen Wipers (Cost=115)
Description:Increases the movement speed bonus of nitro boost.  Perfect for wiping (holo)cats off your windscreen.
	0:Upgrade:Lv1
	0:Movement speed:+20%
	1:Upgrade:Lv2
	1:Movement speed:+40%
Name:Retracting-Spikes Fender (Cost=180)
Description:Adds a damaging knockback effect to the front of your bike when using nitro boost. Hits each target only once.  Quickly gets rid of everyt

Name:Coco Nebulon
Abilities:['Damage Over Time', 'Long-ranged', 'Slow', 'Lifesteal', 'Area of Effect', 'Knockback']
	Health:1300 (2288)
	Movement Speed:8.36
	Attack Type:Melee
	Role:Harasser
	Mobility:Swift
### Skills ###
Skill:Ball Lightning
Coco shoots an actual ball of lightning from her hands, which she can detonate at any time while it's traveling through the air. This little bundle of bolts is able to pass through any solid object.
	Damage=400 (628)
	Cooldown=7s
	Range=8
	Knockback=1.5
	Speed=9.6
### Skill Upgrades ###
Name:Voltage Amplifier (Cost=215)
Description:Increases the base damage of ball lightning against enemy Awesomenauts.  like being hit by Thor himself.
	0:Upgrade:Lv1
	0:Base damage:+18%
	1:Upgrade:Lv2
	1:Base damage:+36%
Name:Gyroscopic Dynamo (Cost=160)
Description:Reduces cooldown time on ball lightning.  Turns you on!
	0:Upgrade:Lv1
	0:Cooldown:-1.5s
Name:Conducting Gel (Cost=165)
Description:Increases the range of ball lightning.  Greased lightning!
	0:Upgrade:

Skill:Tactical Double Jump
Tactical Double Jump
	Jumps=2
### Skill Upgrades ###
Name:Power Pills Turbo (Cost=130)
Description:Increases maximum health.  Insert pill into rear end of digestive tract.
	0:Upgrade:Lv1
	0:Health:+8%
	1:Upgrade:Lv2
	1:Health:+16%
Name:Med-i'-can (Cost=90)
Description:Automatically regenerate health.  Hello... anyone there? Please get me out of here!!!
	0:Upgrade:Lv1
	0:Regeneration:+90%
	1:Upgrade:Lv2
	1:Regeneration:+180%
Name:Space Air Max (Cost=135)
Description:Increases movement speed.  Fashionable and Fast.
	0:Upgrade:Lv1
	0:Movement:Varies per 'naut
Name:Wraith Stone (Cost=120)
Description:Heal additional health by killing critters.  Life sucks, death even more.
	0:Upgrade:Lv1
	0:Heal:300 (528)
Name:Piggy Bank (Cost=0)
Description:Gives 100 Solar.  This product was brought to you by Zork industries, exploiting Zurians since 2780.
	0:Upgrade:Lv1
	0:Solar:+100
Name:Baby Kuri Mammoth (Cost=110)
Description:Reduces the effect of all debuffs  "LOOK!!! A FLY

...

	0:Stun Duration:0.3s
Name:Chunk of Salted Meat (Cost=195)
Description:Increases the damage of Bash.  Ingredients
	0:Upgrade:Lv1
	0:Damage:+10%
	1:Upgrade:Lv2
	1:Damage:+20%
Name:Flaming Fists (Cost=190)
Description:Increases the range of the 3rd Bash combo hit.  I'm on fiya!!
	0:Upgrade:Lv1
	0:Range:+2
Name:Fistbump (Cost=120)
Description:Deflect  bullets with the first two punches of Bash.  Come at me bro!
	0:Upgrade:Lv1
	0:Deflect:Yes
Name:Pale Mead (Cost=350)
Description:Increases the damage the third punch of Bash.  Goes great with some dried meat.
	0:Upgrade:Lv1
	0:3rd punch damage:+24%
Skill:Earthquake
Putting on some weight made his buns of steel a force to be reckoned with. Planting them firmly on the ground after a daring leap shatters earth and enemies alike!
	Damage over time=350 (549.5)
	Damage Duration=5s
	Cooldown=8s
	Range=9
	Time=1s
### Skill Upgrades ###
Name:Electric Hammer (Cost=200)
Description:Adds a snaring effect to Earthquake.  Knock, knock! Who's there?
	0:Upg

Name:Smiles
Abilities:['Tether', 'Lifesteal', 'Knockback', 'Stun']
	Health:1600 (2816)
	Movement Speed:7.6
	Attack Type:Short-ranged
	Role:Fighter
	Mobility:Balanced
### Skills ###
Skill:Trapper's Hook
Throw out a hook that will connect with enemy Awesomenauts and lets you drag them around the map for a short duration.
	Damage=100 (157)
	Duration=1.1s
	Reel in time=2s
	Cooldown=8s
### Skill Upgrades ###
Name:Dinosaur Hook (Cost=115)
Description:Increases the damage of Trapper's Hook.  For big poultry hunting.
	0:Upgrade:Lv1
	0:Damage:+50%
	1:Upgrade:Lv2
	1:Damage:+100%
Name:Electric Magnetic Winch (Cost=185)
Description:Increases the range of the hook  Normally this winch can be found on planetary vehicles.
	0:Upgrade:Lv1
	0:Range:+20%
Name:Candy Rope (Cost=180)
Description:Reduces the 'naut damage output during the hook.  Sweetened rope to lure hungry green creatures.
	0:Upgrade:Lv1
	0:Damage:-20%
Name:Bush Telly (Cost=180)
Description:Increases the duration of Trapper's Hook.  Nothin

Name:Official Conchball (Cost=125)
Description:Increases the size of shell bombs.  Comes with a certificate of authenticity from the intergalactic federation of conchballers.
	0:Upgrade:Lv1
	0:Size:+50%
Name:Spiky Shellball (Cost=180)
Description:Increases the explosion damage of the shell bombs.  The training ball for young Scargi conchballers has sharp spikes to teach them not to touch the ball, but to use a certified broom.
	0:Upgrade:Lv1
	0:Explosion damage:11%
	1:Upgrade:Lv2
	1:Explosion damage:22%
Name:Disco Clamball (Cost=175)
Description:Adds a healing effect to shell bombs for allied Awesomenauts.  The ball can be adjusted with tiny weights on the inside and an adjustable discoball with led lights for extra coolness.
	0:Upgrade:Lv1
	0:Healing:50 (78.5)
	1:Upgrade:Lv2
	1:Healing:100 (157)
Name:Snail Slime Container (Cost=210)
Description:Grants a shield while shellbombs are active.  "A perfect game is a very slow game!" Rob Bourgogne - Sports Caster
	0:Upgrade:Lv1
	0:Shield:20%

### Skill Upgrades ###
Name:Bag Full of Gold Fish (Cost=150)
Description:Deal extra damage with the dive when you have 150 solar in your pockets  Common currency on Okeanos.
	0:Upgrade:Lv1
	0:Damage:+20%
Name:Dead Seahorse Head (Cost=140)
Description:Shoot extra spikes before you dive.  The Omean give these to their children to warn them of the dangers of the seas.
	0:Upgrade:Lv1
	0:Extra Shot:Yes
	0:Extra Spike Damage:18%
	1:Upgrade:Lv2
	1:Extra Shot:Yes
	1:Extra Spike Damage:36%
Name:Alien Abduction Kit (Cost=165)
Description:Adds a lifesteal effect to spike dive  It's probing time!
	0:Upgrade:Lv1
	0:Lifesteal:+30%
Name:Antique Machinegun (Cost=175)
Description:Adds a silence effect to spike dive  This rusty weapon has quite the recoil.
	0:Upgrade:Lv1
	0:Silence:+1s
Name:Chrome File (Cost=210)
Description:Increases damage of spike dive against enemy Awesomenauts.  For male pufferfish hygenic use and escaping prison.
	0:Upgrade:Lv1
	0:Damage:+12%
	1:Upgrade:Lv2
	1:Damage:+24%
Name:Rig

Name:Energy Drink (Cost=140)
Description:Increases the base health of Healbot.  Keep your clone brain healthy and alive... for a while.
	0:Upgrade:Lv1
	0:Base health:+50%
	1:Upgrade:Lv2
	1:Base health:+100%
Name:Overheal Potion (Cost=170)
Description:Increases the base healing power of the Healbot.  Chug! Chug! Chug!
	0:Upgrade:Lv1
	0:Base heal:+18%
	1:Upgrade:Lv2
	1:Base heal:+36%
Name:Turret Add-on (Cost=160)
Description:Causes Healbot to damage nearby enemies.  Desperate diseases call for desperate remedies.
	0:Upgrade:Lv1
	0:Damage:180 (282.6)
	1:Upgrade:Lv2
	1:Damage:360 (565.2)
Name:Hydraulic Sugar Dispenser (Cost=200)
Description:Reduces cooldown of Healbot  Sweet for your friends, sour for your enemies.
	0:Upgrade:Lv1
	0:Cooldown:-2.5s
Name:Cortex Tank (Cost=140)
Description:Healbot will knock away enemies.  Smack that droid up.
	0:Upgrade:Lv1
	0:Knockback:+1.125
	0:Stun duration:+0.1s
Name:Bombshell (Cost=150)
Description:Healbot will leave a mine.  This bomb brings all the bo

In [129]:
fp=open('json.txt','w')
json.dump(awesome_nauts,fp)
fp.close()

# Using the JSON Data we have scraped and saved to file

In [130]:
import jsontree

In [131]:
# Load in the JSON file that contains all of the data
fp=open('json.txt','r')
data = json.load(fp)
fp.close()

data = jsontree.jsontree(character_dict)
char_name = data.get("name")

In [132]:
skill = data.get("skills")[0]
skill_name = skill.get("name")
print(f"{char_name}:{skill_name}:-")
for upgrade in skill.get("upgrades"):
    upgrade_name = upgrade.get("name")
    upgrade_description = upgrade.get("description")
    upgrade_img = upgrade.get("img")
    print(f"\n* {upgrade_name} - {upgrade_description}")
    
    upgrade_levels = upgrade.get("levels")
    upgrade_name = upgrade.get("name")
    
    df = pd.DataFrame(upgrade_levels)
    print(df.T)
    '''
    for level in upgrade_levels:
        for k,v in level.items():
            print(f"\t\t{k}={v}")
    '''

Yuri:Mine Deploying:-

* Titanium Spikes - Increases the base damage of mines.  Arm your mines to the teeth.
                0     1
Upgrade       Lv1   Lv2
Base damage  +10%  +20%

* Toaster Timer - Increases the longevity of mines.  We like our enemies extra crispy.
           0    1
Upgrade  Lv1  Lv2
Time     +2s  +4s

* Aerial Spring - Makes mines bounce up and down.  a.k.a. The Dove Exterminator
            0
Upgrade   Lv1
Bounce    Yes
Gravity    x2
Speed    300%

* Mine Constructor - Reduces cooldown time on mines.  By warping the mines directly from our factory to you, we can provide the quickest recharge time.
              0    1
Upgrade     Lv1  Lv2
Cooldown  -0.5s  -1s

* Carpet Bombs - Increases the amount of mines placed, dividing total damage evenly across them.  Instructions
             0
Upgrade    Lv1
Damage    -60%
Mines       +2
Size     66.7%

* Uranium Spikes - Exploded mines become healthpacks.  My spikes are bigger than yours.
                                0 

In [133]:
df = pd.DataFrame(upgrade_levels).T

In [134]:
print(df)

                                0               1
Upgrade                       Lv1             Lv2
Healthpack                    Yes             Yes
Heal                    100 (157)       200 (314)
Duration           50% (of mines)  50% (of mines)
Carpet bombs heal             60%             60%


# Do some formatting to HTML

In [135]:
# Load in the JSON file that contains all of the data
fp=open('json.txt','r')
data = json.load(fp)
fp.close()

# Create a JSON tree to allow easy naviagtion of the data in the file
tree = jsontree.jsontree(data)

In [136]:
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader
import IPython

In [155]:
# Load in the template to be used for each Flash card
templateLoader = FileSystemLoader(searchpath=".")
env = Environment(loader=templateLoader)
template = env.get_template("card_template.html")

In [156]:
# Play around to see if the data looks ok...
#print(tree.keys())
character = tree.get('Admiral Swiggins')
#print(character)
skills = character.get('skills')
#print(skills)
skill = skills[0]
upgrades = skill.get('upgrades')
upgrade = upgrades[0]
levels = upgrade.get('levels')
skill

{'name': 'Anchor Hook',
 'description': "Swiggins hurls Homboldt forward, and grapples towards whatever he hits. If you thought getting hit by an anchor is bad, wait for the iron-clad squid that's coming after it!",
 'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/0/0b/UI_Skillbutton_Captain_AnchorHook.png',
 'stats': {'Damage': '310 (486.7)',
  'Droid damage': '100 (157)',
  'Stun Duration': '0.25s',
  'Cooldown': '7s',
  'Range': '11'},
 'upgrades': [{'name': 'Flying Fish Compass',
   'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/e/e3/Shop_icons_captain_skill_b_upgrade_a.png',
   'description': 'Reduces the cooldown of anchor hook  In the winter they point south, in the summer they point north, easy!',
   'cost': '190',
   'levels': [{'Upgrade': 'Lv1', 'Cooldown': '-2s'}]},
  {'name': 'Blue Heart Medal',
   'img': 'https://static.wikia.nocookie.net/awesomenauts_gamepedia/images/d/de/Shop_icons_captain_skill_b_upgrade_c.png',
   'descr

## Create the Flash cards

In [158]:
# Create HTML Flash cards for every characters skill uopgrades

# HTML output
output = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Awesomenauts Skill Upgrades</title> \
    <link rel="stylesheet" href="template.css"></head><body>'

# Loop through all of teh characters in the JSON tree...
for character_name in tree.keys():
    
    #character_name = "Yuri"
    
    # Get a specific character from the tree
    character = tree.get(character_name)
    
    # Get their skills list
    skills = character.get('skills')

    # Loop through all of their skills...
    for skill in skills:
        
        skill_stats = ""
        stats = skill.get('stats')
        for k,v in stats.items():
            skill_stats += f"{k}:{v}<br>"
            
        
        # Get the list of upgrades the this skill
        upgrades = skill.get('upgrades')
        
        # Loop thourh all of the upgrades
        for upgrade in upgrades:

            # Get the basic details of the upgrade
            upgrade_name = upgrade.get('name')
            cost = upgrade.get('cost')
            
            # Get the effect of the upgrade at different levels
            levels = upgrade.get('levels')

            # Array to store 1-2 level descriptions for this upgrade
            level_descriptions = [f"Upgrade Lv{i+1} : None" for i in range(len(levels)+1)]
            
            # loop through the levels of effect for this upgrade
            for i,level in enumerate(levels):
                
                # Build a description of each level effect
                level_description = ""
                
                # Loop through each effect
                for ii,kv in enumerate(level.items()):
                    k,v=kv
                    level_description += f"{k} {v}"
                    if ii == 0:
                        level_description += ": "
                    elif ii < len(level)-1:
                        level_description += ", "
                
                level_descriptions[i] = level_description

            # Pass all of the data into the renderer to make an html card
            html = template.render(name = character.get("name"),
                                   skill = skill.get("name"),
                                   skill_stats = skill_stats,
                                   upgrade = f"{upgrade_name} [${cost}]",
                                   description = upgrade.get("description"),
                                   image = upgrade.get("img"),
                                   skill_image = skill.get("img"),
                                   level1 = level_descriptions[0],
                                   level2 = level_descriptions[1],
                                  )


            # Add the card to the complete list
            output += html
            
    # Just do one character for now
    #break
    output += "<div></div>"
    
output+= "</body></html>"
    
IPython.display.HTML(output)


In [159]:
#print(output)

In [160]:
# Write the HTML output string to a file

fp=open('nauts.html','wb')
fp.write(output.encode('utf-8'))
fp.close()